In [1]:
# %%
#  các thư viện
# pip install -r requirements.txt

# %%
import json
# from collectionamespace import defaultdict, Counter, deque
import mwparserfromhell  
import xml.etree.ElementTree as ET
from tqdm import tqdm
import re

# %%
# TẬP HẠT GIỐNG
SEED_LIST_FILE = "seed_list.txt"

XML_FILE = "viwiki-latest-pages-articles.xml"

# %% [markdown]
# # ĐỌC FILE TẬP HẠT GIỐNG

# %% [markdown]
# ## read_seeds

# %%
def read_seeds(path):
    '''Đọc danh sách từ file, mỗi dòng là 1 trang wikipedia.'''
    seed_list = []
    with open(path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:  # Bỏ qua dòng trống
                seed_list.append(line)
    return seed_list

# test

print(read_seeds(SEED_LIST_FILE))

# %% [markdown]
# # ĐỌC TEXT TỪ FILE XML WIKIPEDIA

# %%



# %%
# đọc file XML wiki dump 

# mỗi bài viết trong thẻ <page> </page> 

    # Trả về danh sách [(title, text), ...]

# dùng iterparse (đọc từng phần nhỏ)


def read_pages_from_xml(xml_path):
    """
    Đọc từng trang trong file XML (Wikipedia dump).
    Trả về danh sách [(title, text), ...]
    """
    # danh sách lưu kết quả
    pages = []  
    # iterparse => đọc
    context = ET.iterparse(xml_path, events=("start", "end"))

    # Lấy namespace (ở thẻ <mediawiki>)
    event, root = next(context)
    
    if root.tag.startswith("{"):
        namespace = root.tag.split("}")[0].strip("{")
        namespace_prefix = f"{{{namespace}}}"
    else:
        namespace_prefix = ""

    
    # Duyệt từng p.tử trg XML
    for event, element in context:
        if event == "end" and element.tag == namespace_prefix + "page":
            title_element = element.find(f"./{namespace_prefix}title")
            text_element = element.find(f".//{namespace_prefix}text")
            
            # Lấy nd trg các thẻ, nếu k có thì None 
            if title_element is not None:
                title = title_element.text
            else:
                title = None

            if text_element is not None:
                text = text_element.text
            else:
                text = ''

            pages.append((title, text))  # thêm vào danh sách

            element.clear()  # xóa khỏi bộ nhớ

    return pages






# %%
all_pages = read_pages_from_xml(XML_FILE)

wiki_dict = {}
for title, text in all_pages:
    if title:
        wiki_dict[title] = text

def get_wikitext(title):
    """
    Lấy nội dung wikitext từ title
    k tìm thấy, trả về chuỗi rỗng
    """
    if title not in wiki_dict: return ""
    
    wikitext = wiki_dict[title]
    if wikitext.startswith("#đổi") or wikitext.startswith("#redirect"):
        start = wikitext.find('[[')
        end = wikitext.find(']]')
        if start != -1 and end != -1:
            new_title = wikitext[start+2:end].strip()
            return get_wikitext(new_title)

    return wikitext


# %% [markdown]
# ## generate_date_filename

# %%
from datetime import datetime

def generate_date_filename():
    now = datetime.now()
    filename = now.strftime("%d-%m-%Y_%H-%M-%S")
    return filename



# with open("test/test_wikitext.txt", "w", encoding="utf-8") as f:
#     f.write(get_wikitext("Ninh Dương Lan Ngọc"))

# %% [markdown]
# ## chung

# %%
#chung

def save_to_txt(content ,file_name = None, auto_name = False):
    if auto_name or file_name is None:
        file_name = generate_date_filename() 
         
    with open(f"test/{file_name}.txt", "w",  encoding="utf-8") as f:
        f.write(content) 

def save_to_json(content ,file_name = None,auto_name = False):
    if auto_name or file_name is None:
        file_name = generate_date_filename()
    with open(f"test/{file_name}.json", "w",  encoding="utf-8") as f:
        json.dump(content, f, ensure_ascii=False, indent=4)


def print_json_format(res):
    print(json.dumps(res, ensure_ascii=False, indent=4))

def df_to_dict(df, orient='records'):
    return df.to_dict(orient=orient), df.columns.tolist()

# %% [markdown]
# # TRÍCH XUẤT THÔNG TIN CƠ BẢN

# %% [markdown]
# ## extract_infobox_from_wikitext

# %%
def extract_infobox_from_wikitext(wikitext):
    
    if not wikitext:
        return {}
    try:
        # biến chuỗi wikitext => Wikicode obj
        code = mwparserfromhell.parse(wikitext)

    except Exception:
        return {}
    
    # duyệt qua toàn bộ cấu trúc wikitext, trả về list all các template (các khối {{ ... }})
    templates = code.filter_templates() 
   
    info = {}

    for template in templates:
        # template là obj

        # tên của template 
        name = str(template.name).lower()
        infoboxs = ["nhân vật", "phim", "diễn viên", "nghệ sĩ",  
                    "film", "truyền hình", "điện ảnh",
                    "television" , "person"]

        for infobox in infoboxs:
            if infobox in name.lower():
                # iterate params. in list tham so
                for param in template.params:
                    key = str(param.name).strip()
                    value = str(param.value).strip()
                    if key and value:
                        info[key] = value
                # break nếu k rỗng k false 
                if info:
                    break
    return info

# %% [markdown]
# ## check_type_of_infobox

# %%
def check_type_of_infobox(wikitext):
    if not wikitext:
        return {}
    try:
        # biến chuỗi wikitext => Wikicode object
        code = mwparserfromhell.parse(wikitext)

    except Exception:
        return {}
    
    templates = code.filter_templates() 

    # thông tin nhân vật => nhân vật
    person_keys = [
        "nhân vật", "diễn viên", "nghệ sĩ", "person"
    ]
    film_keys = [
        "phim", 
        "film",
        "truyền hình",
        "điện ảnh",
        'television'
    ]
    for template in templates:
        name = str(template.name).lower()
 
        for key in film_keys:
            if key in name.lower(): return "film"
        for key in person_keys:
            if key in name.lower(): return "person"
    return ""




# %%
#test
ndln_wikitext = get_wikitext("Ninh Dương Lan Ngọc")


#test 

matBiec_wikitext = get_wikitext("Mắt biếc (phim)")
print(matBiec_wikitext)
# save_to_txt(matBiec_wikitext,"test/test_film_wikitext")

matBiec_infobox = extract_infobox_from_wikitext(matBiec_wikitext)
print("type mat biec infobox",type(matBiec_infobox))
# save_to_json( matBiec_infobox,"test/test_film_infobox")


# %% [markdown]
# Wikipedia không nói rõ đâu là phim, đâu là người
# chỉ tiêu đề + cú pháp [[...]]
# 
# phân loại liên kết -> người / phim

# %% [markdown]
# ## is_vn_film_wikilink

# %%
# check xem 1 link là phim or k 
def is_vn_film_wikilink(link_title):
    if link_title is None or link_title.strip() == "":
        return False   
     
    # tức là link này k có dấu hiệu nhận biết tại tiêu đề
    # get text ra để check infobox 
    wikitext = get_wikitext(link_title)
    if (check_type_of_infobox(wikitext) == "film"):
        infobox = extract_infobox_from_wikitext(wikitext)
        infobox = {k.lower(): v for k, v in infobox.items()}

        country = None
        language = None

        # Lấy giá trị country / quốc gia
        for key in ['country', 'quốc gia']:
            if key in infobox:
                country = str(infobox[key]).lower()
                break

        # Lấy giá trị language / ngôn ngữ
        for key in ['language', 'ngôn ngữ']:
            if key in infobox:
                language = str(infobox[key]).lower()
                break

        # Ưu tiên country
        if country:
            if any(x in country for x in ['{{vie}}', '{{vn}}', '{{vnm}}', 'việt nam']):
                return True  # Quốc gia là Việt Nam => phim Việt Nam
            else:
                return False  # Quốc gia khác => không phải phim Việt Nam

        # Nếu không có country => dùng language để đoán
        if language and 'tiếng việt' in language:
            return True
        else:
            return "chưa rõ"
                
    

#test 
# is_vn_film_wikilink
# check_type_of_infobox
print(is_vn_film_wikilink("Cô gái từ quá khứ"))
print(is_vn_film_wikilink("Cánh đồng bất tận (phim)"))
print(is_vn_film_wikilink('Kong: Đảo Đầu lâu'))
print(is_vn_film_wikilink('Long Ruồi'))

print(is_vn_film_wikilink('Tèo em'))
print(is_vn_film_wikilink('Tấm Cám: Chuyện chưa kể'))

x = 'Vừa đi vừa khóc'
print(is_vn_film_wikilink(x))

x = 'Cô nàng bướng bỉnh'
print(is_vn_film_wikilink(x))



# %%
#test
print(check_type_of_infobox(x))
x1 = get_wikitext(x)
x2 = extract_infobox_from_wikitext(x1)
print(x2)

# %%
def df_to_filted_vn_film_dict(df):
    dicts, cols = df_to_dict(df)

    film_key = cols[1] # cột thứ 2: tựa phim
    filtered = []
    for d in dicts:
        value = d.get(film_key)
        if is_vn_film_wikilink(value):
            filtered.append(d)
    
    return filtered

# %% [markdown]
# ## is_vn_person_wikilink

# %%
# check 1 link la person or k

def is_vn_person_wikilink(link_title):
    if not link_title or not link_title.strip():
        return False

    wikitext = get_wikitext(link_title)
    if not wikitext:
        print(link_title, "ko có wikitext")
        return False

    # Không phải nhân vật, loại 
    if check_type_of_infobox(wikitext) != "person":
        print(link_title, "check_type_of_infobox không là person")
        return False

    # Lấy INFObox
    infobox = extract_infobox_from_wikitext(wikitext)
    infobox_lower = {k.lower(): str(v).lower() for k, v in infobox.items()}
    vn_keys = ["việt nam", "{{vie}}", "vietnam" ,"{{vnm}}"]

    # quốc tịch 
    for key in ["quốc tịch", "nationality"]:
        if key in infobox_lower:
            value = infobox_lower[key]
            if any(v in value for v in vn_keys):
                return True
            else:
                print(link_title, "quốc tịch KHÔNG phải VN → loại")
                return False

    # check toàn bộ value
    text_all = " ".join(infobox_lower.values()).lower()
    if any(v in text_all for v in vn_keys):
        return True

    # check template name 
    code = mwparserfromhell.parse(wikitext)
    template_names = " ".join(str(t.name).lower() for t in code.filter_templates())
    if any(v in template_names for v in vn_keys):
        return True
 
    if "người việt nam" in wikitext.lower().strip() or "người việt" in wikitext.lower().strip():
        print(link_title,"ko rõ quốc tịch, nhưng mô tả là người vn")
        return True

    # trang định hướng 
    if 'có thể' in wikitext or 'định hướng' in wikitext:
        print(link_title, "ko xác định đc wiki page (định hướng)")
        return False

    print(link_title, "không là người VN")
    return False


#test

# print(is_vn_person_wikilink("Ninh Dương Lan Ngọc"))
# print(is_vn_person_wikilink("Trần Nghĩa (diễn viên)"))
# print(is_vn_person_wikilink("Trúc Anh"))
# print(is_vn_person_wikilink("Victor Vũ"))
# print(is_vn_person_wikilink("Thanh Hiền"))
# print(is_vn_person_wikilink("Taylor Swift"))
print(is_vn_person_wikilink("Lương Thế Thành"))
print(is_vn_person_wikilink("Huy Cường"))
print(is_vn_person_wikilink("Hồng Thy"))

# %%
# print(get_wikitext("Trúc Anh"))
print(check_type_of_infobox(get_wikitext("Trúc Anh")))


# %% [markdown]
# ## unique_list

# %%
#chung
# Bỏ lặp, giữ đúng thứ tự 

def unique_list(items_list):

    seen_items = set()   # chứa các phần tử đã gặp
    result = []          # danh sách kết quả

    for item in items_list:
        if item not in seen_items:
            result.append(item)    # thêm vào kết quả
            seen_items.add(item)   # đánh dấu là đã gặp

    return result


#test 
print(unique_list(["Trấn Thành", "Ngô Thanh Vân", "Trấn Thành"]))

# %% [markdown]
# ## parse_heading

# %%
# == Danh sách phim == => {'level': 2, 'title': 'Danh sách phim'}
def parse_heading(line):
    line = line.strip() # clear space đầu cuối

    # k '=' ở đầu, cuối => pass
    if not line.startswith('=') or not line.endswith('='):
        return None  # không phải tiêu đề wiki

    # Đếm số '=' ở đầu và cuối
    left_eq = len(line) - len(line.lstrip('='))
    right_eq = len(line) - len(line.rstrip('='))

    if left_eq != right_eq: # ==abc===
        return None  # không cân đối, bỏ qua

    # Cắt phần tên tiêu đề ở giữa
    title = line[left_eq:len(line) - right_eq].strip()
    if not title: return None # k content 

    return {
        'level': left_eq, # số lượng '=' --> cấp độ title
        'title': title
    }

# Thử
lines = [
    "== Danh sách phim ==",
    "=== Điện ảnh ===",
    "==== Truyền hình ====",
    "Không phải tiêu đề",
]

for l in lines:
    result = parse_heading(l)
    if result:
        print(result)


# %% [markdown]
# ## is actor, is director

# %%
def is_actor(infobox_dict):
    # lay content của 2 cái ghép thàn 1 chuỗi, check keyword trong chuỗi => true
    job = (infobox_dict.get('công việc', '') 
           + infobox_dict.get('nghề nghiệp', '')
            + infobox_dict.get('occupation', '')
           ).lower()
    return 'diễn viên' in job

def is_director(infobox_dict):
    # true if nha lam phim
    job = (infobox_dict.get('công việc', '') 
           + infobox_dict.get('nghề nghiệp', '')
            + infobox_dict.get('occupation', '')
           ).lower()
    return 'nhà làm phim' in job

t0 = "Victor Vũ"
t1 = extract_infobox_from_wikitext(get_wikitext(t0))
print(t0,'--> actor:', is_actor(t1),'--> director', is_director(t1))
t0 = "Trấn Thành"
t1 = extract_infobox_from_wikitext(get_wikitext(t0))
print(t0,'--> actor:', is_actor(t1),'--> director', is_director(t1))
t0 = "Ngô Thanh Vân"
t1 = extract_infobox_from_wikitext(get_wikitext(t0))
print(t0,'--> actor:', is_actor(t1),'--> director', is_director(t1))
t0 = "Charlie Nguyễn"
t1 = extract_infobox_from_wikitext(get_wikitext(t0))
print(t0,'--> actor:', is_actor(t1),'--> director', is_director(t1))
t0 = "Lý Hải"
t1 = extract_infobox_from_wikitext(get_wikitext(t0))
print(t0,'--> actor:', is_actor(t1),'--> director', is_director(t1))
t0 = "Jun Vũ"
t1 = extract_infobox_from_wikitext(get_wikitext(t0))
print(t0,'--> actor:', is_actor(t1),'--> director', is_director(t1))


# %% [markdown]
# ## extract_films_section

# %%
# extract_films_section from wiki person 

def extract_films_section(wikitext,depth=0):
  

    # check là actor or director 
    infobox_dict = extract_infobox_from_wikitext(wikitext)
    type_node = check_type_of_infobox(wikitext)

    if type_node == 'film': return "==> ko extract films từ wiki film"
    if depth == 0 and is_actor(infobox_dict) == False and is_director(infobox_dict) == False: 
        return "person ko là actor cũng ko là director"
    
    """
    Tách phần wikitext thuộc tiêu đề cấp 2 (== ... ==)
    có 'phim' hoặc 'film' trong tiêu đề.
    Giữ nguyên wikitext gốc, chỉ so khớp tên tiêu đề ở dạng thường.
    Nếu section chỉ chứa liên kết {{chính|...}} thì đệ quy sang trang đó.
    """
    if not wikitext or depth > 2: return ""

    lines = wikitext.splitlines()

    headings = []  # lưu (index_dòng, heading_dict)

    for i, line in enumerate(lines):
        h = parse_heading(line)
        if h:
            headings.append((i, h))

    # chọn heading cấp 2 có từ "phim"/"film"
    chosen = None
    for i, h in headings:
        title_lower = h['title'].lower()
        if h['level'] == 2 and ('danh sách phim' in title_lower or 'phim' in title_lower or 'film' in title_lower):
            chosen = (i, h)
            break
        elif h['level'] == 2 and ('điện ảnh' in title_lower):
            chosen = (i, h)  
            # (51, {'level': 2, 'title': 'Sự nghiệp điện ảnh'})
            break
        elif h['level'] == 2 and ('sự nghiệp' in title_lower):
            chosen = (i, h)
            
            

    if not chosen: return ""

    start_line_index = chosen[0]
    chosen_level = chosen[1]['level'] # 2 => level 2 

    # tìm heading cấp 2 kế tiếp để biết điểm kết thúc
    end_line_index = len(lines)
    for i, h in headings:
        if i > start_line_index and h['level'] == chosen_level:
            end_line_index = i
            break

    # trích phần nội dung giữa hai heading
    section = "\n".join(lines[start_line_index + 1:end_line_index]).strip()
    
    # --- kiểm tra nếu chỉ là liên kết {{chính|...}} hoặc {{main|...}} ---
    main_re = re.search(r"\{\{\s*(?:chính|main)\s*\|\s*([^}|]+)", section, flags=re.IGNORECASE)
    if main_re:
        linked_title = main_re.group(1).strip()
        try:
            linked_wikitext = get_wikitext(linked_title)
            if linked_wikitext:
                # đệ quy sang trang phụ, tăng depth
                return extract_films_section(linked_wikitext, depth=depth + 1)
        except Exception as e:
            print("Lỗi khi lấy wikitext:", e)
    return section

# test
# print(extract_films_section(get_wikitext("Trấn Thành")))
# print(extract_films_section(get_wikitext("Victor Vũ")))
# print('================================')
# print(extract_films_section(get_wikitext("Nguyễn Hòa Bình (doanh nhân)")))
# print('================================')
# print(extract_films_section(get_wikitext("Lý Hải")))
# print('================================')
# print(extract_films_section(get_wikitext("Charlie Nguyễn")))
print('================================')
print(extract_films_section(get_wikitext("Jun Vũ")))

# %% [markdown]
# ## extract_films_from_subsection

# %%
def extract_films_from_subsection(film_section_wikitext):
    if not film_section_wikitext:
        return []

    lines = film_section_wikitext.splitlines()
    headings = []
    results = []
    # --- Thu thập các heading ---
    for i, line in enumerate(lines):
        h = parse_heading(line)
        if h:
            headings.append((i, h))

    # --- Lọc các heading level 3 ---
    level3_heads = [(i, h) for i, h in headings if h["level"] == 3]

    # --- k có heading cấp 3: toàn bộ là 1 phần (nếu có bảng) ---
    if not level3_heads:
        print("extract_films_from_subsection: Section không có heading cấp 3")
        if "{|" in film_section_wikitext:
            results.append({
                "title": "Phim",
                "subsection_wikitext": film_section_wikitext.strip()
            })
        return results

    # --- Có heading cấp 3: xử lý phần trc heading đầu tiên ---
    first_start = level3_heads[0][0]
    # lines là danh sách từng dòng (splitlines() cần nối trc heading 
    before_first = "\n".join(lines[:first_start]).strip()
    if before_first and before_first.startswith("{|"):
        results.append({
            "title": "Phim",
            "subsection_wikitext": before_first
        })
    # --- xử lý từng subsection có heading cấp 3 --- 
    for idx, (start_line, h) in enumerate(level3_heads):
        # xác định end_line là heading kế tiếp hoặc hết file
        end_line = len(lines)
        if idx + 1 < len(level3_heads):
            end_line = level3_heads[idx + 1][0]

        subsection = "\n".join(lines[start_line + 1:end_line]).strip()
        results.append({
            "title": h["title"].strip(),
            "subsection_wikitext": subsection
        })

   
    
    return results

#test
b0 ="Trấn Thành"
b1 = "Ninh Dương Lan Ngọc"
b2 = "Ngô Thanh Vân"
b2 = "Charlie Nguyễn"
b2 = "Jun Vũ"
a0 = b2
a1 = (extract_films_section(get_wikitext(a0)))
# print(a1)
a2= extract_films_from_subsection(a1)
print_json_format(a2)


# %% [markdown]
# ## extract_headers

# %%
def extract_headers(wikitext):
    """
    Trích xuất header hoàn chỉnh từ bảng wikitext (2 dòng header).
    Nếu có ! colspan="n" | HeaderCha thì bỏ HeaderCha, thay bằng n header con dòng dưới.
    """
    wikitext = re.sub(r'style\s*=\s*["\'].*?["\']', '', wikitext)

    lines = wikitext.strip().split('\n')
    header_lines = []
    
    # Lấy tất cả dòng header (bắt đầu bằng !)
    for line in lines:
        line = line.strip()
        if line.startswith('!'):
            header_lines.append(line)
       
    # print('@1__', header_lines)
    if not header_lines:
        return []

    # Xử lý từng dòng thành list các ô
    headers_res = []
    headers_row2 = []
    header_rowspan = False # mặc định header 1 dòng thôi, True là 2 dòng 

    if "rowspan" in header_lines[0]:
        header_rowspan = True
    
    hash_index = None
    for line in header_lines:
        if header_rowspan:
            # print('@2__',line) #! rowspan="2" |Năm
            # bỏ ! đi đã 
            line = line.replace("!",'').strip()
            if line.startswith('rowspan'):
                parts = line.split('|')
                # ['rowspan="2" ', 'Năm']
                headers_res.append(parts[1].strip())
                continue
            elif line.startswith('colspan'):
                headers_res.append('#')
                continue
            else:
                headers_row2.append(line.strip())

        else: 
            if line.startswith("!"):
                line = line[1:] # xóa kí tự đầu 
                columns = line.split('!!')
                for c in columns: headers_res.append(c)
                continue # add xong thì bỏ qa    
            
    for i, item in enumerate(headers_res):
        if item == '#':
            hash_index = i
            break   # nếu chỉ muốn vị trí đầu tiên của '#'
            
    if hash_index is not None:
        headers_res[hash_index:hash_index] = headers_row2 # chèn trước #
        headers_res.remove('#')  # xóa ký tự '#'

    return(headers_res)


#test
wikitext1 = """
|- style="background:#ccc; text-align:center;" 
! rowspan="2" style="width:33px;"|Năm
! rowspan="2" | Tựa đề
! colspan="2" |Vai trò
! rowspan="2" |Vai diễn
! rowspan="2" |Ghi chú
|-
! Đạo diễn
! Sản xuất
|-
"""

wikitext = '''
{| class="wikitable"
!Năm!!Tựa phim!!Vai diễn!!Ghi chú
|-
'''
print(extract_headers(wikitext1))
print(extract_headers(wikitext))

print(extract_headers(a2[0]['subsection_wikitext']))

# %% [markdown]
# ## clean_wiki_markup

# %%
def clean_wiki_markup(text):
    code = mwparserfromhell.parse(text)

    # Nếu có wikilink [[...]]
    links = code.filter_wikilinks()
    if links:
        return str(links[0].title).strip()

    # Nếu có template {{...}}
    templates = code.filter_templates()
    if templates:
        return str(templates[0].name).strip()

    # Nếu không có gì đặc biệt
    return code.strip_code().strip()

print(clean_wiki_markup("[[Ngô Thanh Vân]] là [[đạo diễn]]"))
print(clean_wiki_markup('[[Ninh Dương Lan Ngọc]]'))
print(clean_wiki_markup('{{Không}}'))
print(clean_wiki_markup("''ABC''"))

# %%
def parse_span(token):
    """
    Nhận 1 token dạng 'rowspan="2" | dữ liệu' hoặc chỉ dữ liệu
    Trả về (rowspan, colspan, data)
    """
    rowspan = 1
    colspan = 1
    data = token.strip()
    
    # Kiểm tra xem có rowspan/colspan không
    parts = data.split('|', 1)  # split 1 lần, còn lại là data
    attr = parts[0].strip()
    
    # check rowspan
    if 'rowspan' in attr:
        try:
            # loại bỏ ký tự thừa
            value = ''.join(c for c in attr if c.isdigit())
            rowspan = int(value)
        except:
            pass
        data = parts[1].strip() if len(parts) > 1 else ''
    
    # check colspan
    if 'colspan' in attr:
        try:
            value = ''.join(c for c in attr if c.isdigit())
            colspan = int(value)
        except:
            pass
        data = parts[1].strip() if len(parts) > 1 else ''
    
    return rowspan, colspan, data

# Test
lines = [
    '|2001',
    "|''Hương dẻ''",
    '| {{Không}}',
    '| {{Không}}',
    '|Na lớn',
    '|rowspan="2" |phim truyền hình',
    '|2004',
    "|''Rouge''",
]

for line in lines:
    if line.startswith('|'):
        r, c, d = parse_span(line[1:])
        print(r, c, d)

# %%
#split_top_level_double tách dòng theo || nhưng bỏ qua những || nằm trong [[...]] hoặc {{...}}
# split theo ||
def split_theo_gach_kep(s):
    """
    Split string s on top-level '||' occurrences (ignore those inside [[...]] or {{...}}).
    Returns list of segments.
    """
    segs = []
    buf = []
    i = 0
    n = len(s)
    # nesting counters for [[...]] and {{...}} and also [ and ] and { and }
    brack = 0
    curl = 0
    squote = 0
    while i < n:
        # check for '[[' or ']]', '{{' or '}}'
        if s.startswith('[[', i):
            brack += 1
            buf.append('[[')
            i += 2
            continue
        if s.startswith(']]', i):
            if brack > 0: brack -= 1
            buf.append(']]'); i += 2; continue
        if s.startswith('{{', i):
            curl += 1
            buf.append('{{'); i += 2; continue
        if s.startswith('}}', i):
            if curl > 0: curl -= 1
            buf.append('}}'); i += 2; continue

        # top-level '||' when not inside brackets/templates
        if s.startswith('||', i) and brack == 0 and curl == 0:
            segs.append(''.join(buf).strip())
            buf = []
            i += 2
            continue

        buf.append(s[i])
        i += 1

    segs.append(''.join(buf).strip())
    return segs

def find_top_level_pipe(s):
    """
    Find index of first top-level '|' used as attribute/value separator,
    ignoring '|' inside [[...]] or {{...}}.
    Returns index or -1 if none.
    """
    i = 0
    n = len(s)
    brack = 0
    curl = 0
    while i < n:
        if s.startswith('[[', i):
            brack += 1; i += 2; continue
        if s.startswith(']]', i):
            if brack > 0: brack -= 1
            i += 2; continue
        if s.startswith('{{', i):
            curl += 1; i += 2; continue
        if s.startswith('}}', i):
            if curl > 0: curl -= 1
            i += 2; continue

        # single '|' char (not '||' because those were handled earlier)
        if s[i] == '|' and brack == 0 and curl == 0:
            return i
        i += 1
    return -1

a5 ='[[abc||cvd]]||abc||efd|b'
print(split_theo_gach_kep(a5))

# %% [markdown]
# ## wikitable_to_df

# %%
import mwparserfromhell
import re
import pandas as pd

import re


def wikitable_to_df(wikitext, headers):
    lines = wikitext.strip().split('\n')
    
    # số cột
    num_cols = len(headers)

    table_rows = []          # danh sách các dòng đã parse
    rowspan_tracker = {}     # theo dõi các ô có rowspan còn kéo dài
    
    current_row_index = -1        # chỉ số hàng đang xử lý
    for line in lines:
        line = line.strip()
      
        # {| class wikitable or open table => pass
        if not line or(line.startswith("{|")): continue
        # end of table
        if line.startswith("|}") or line == "}" or line == "|}" :
            break
        # ---Thêm dòng mới trống:
        # ---đại diện cho 1 hàng dữ liệu mới sắp được điền.
        if line.startswith('|-'):
            table_rows.append([None]*num_cols) #grid = [[None, None, None]]
            current_row_index += 1 #0
            # --- Áp dụng các ô rowspan còn hiệu lực cho dòng hiện tại ---
            for key in list(rowspan_tracker.keys()):
                start_row, col = key
                remaining_rows, value = rowspan_tracker[key]

                # Nếu dòng hiện tại nằm sau dòng bắt đầu rowspan
                if current_row_index > start_row and remaining_rows > 0:
                    # Nếu ô ở vị trí này đang trống thì điền giá trị
                    if table_rows[current_row_index][col] is None:
                        table_rows[current_row_index][col] = value

                    # Giảm số dòng còn lại của rowspan đi 1
                    remaining_rows -= 1
                    if remaining_rows <= 0: del rowspan_tracker[key]
                    else: rowspan_tracker[key] = (remaining_rows, value)

            continue
        
        if line.startswith('!'): continue # pass
        # dòng dữ liệu bình thường bắt đầu bằng '|' 
        if (line.startswith('|')):
            # đảm bảo có một hàng hiện tại; else hãy tạo một hàng (xử lý các bảng bỏ qua '|-' ban đầu) 
            if current_row_index == -1: 
                table_rows.append([None] * num_cols) 
                current_row_index = 0
            # xóa | ở đầu dòng đi
            line = line[1:].strip() 
            # chia thành các phân đoạn '||'
            segments = split_theo_gach_kep(line)

            # --- Tìm vị trí cột trống đầu tiên trong hàng hiện tại ---
            row = table_rows[current_row_index]
            # tìm cột trống đầu tiên để bắt đầu chèn , update col_index khi chèn 
            col_index = 0
            while col_index < num_cols and row[col_index] is not None:
                col_index += 1

            for seg in segments:
                if seg == '':
                    # empty cell => treat as empty string
                    rowspan, colspan, data = 1, 1, ''
                else:
                    rowspan, colspan, data = parse_span(seg)
                
                cell_value = clean_wiki_markup(data)
                # Tìm col_index khả dụng tiếp theo trong trường hợp cột trước đã bị chiếm bởi rowspan/colspan.
                while col_index < num_cols and row[col_index] is not None:
                    col_index += 1
                if col_index >= num_cols:
                    # no space left: skip (or break)
                    break
                # gán giá trị cho cell, theo colspan
                for i in range(colspan):
                    if col_index + i < num_cols:
                        row[col_index + i] = cell_value
                # nếu rowspan > 1 -> lưu lại để áp dụng cho các hàng tiếp theo, remaining rows = rowspan - 1
                if rowspan > 1:
                    rowspan_tracker[(current_row_index, col_index)] = (rowspan - 1, cell_value)
                #tăng các col index qa các cell đã đặt 
                col_index += colspan
            continue

        continue

        
            
    
    # --- Tạo DataFrame ---
    # bỏ các hàng toàn None
    table_rows = [r for r in table_rows if not all(v is None for v in r)]
    df = pd.DataFrame(table_rows, columns=headers)
    return df

# --- Sử dụng ---

#test 
b0 ="Trấn Thành"
b1 = "Ninh Dương Lan Ngọc"
b2 = "Ngô Thanh Vân"
a0 = b2
a1 = (extract_films_section(get_wikitext(a0)))
a2= extract_films_from_subsection(a1)
# print_json_format(a2)

# Tự lấy headers
headers = extract_headers(a2[0]['subsection_wikitext'])
print("Headers:", headers)

# Parse table
df = wikitable_to_df(a2[0]['subsection_wikitext'], headers)
print(df)
save_to_txt(str(df), auto_name = True)


# %% [markdown]
# ## extract_films_from_person_wikitext

# %%
def extract_films_from_person_wikitext(wikitext):
    '''
    wikitext - all
    wikitext - section chứa mọi phim
    wikitext - section từng loại phim 
        [
            {
                "title": "Phim điện ảnh",
                "subsection_wikitext": "{| class=\"wikitable\"\n|- style=\"background:#ccc; text-align:center;\"\n! rowspan=\"2\" style=\"width:33px;\"|Năm\n! rowspan=\"2\" | Tựa đề\n! colspan=\"2\" |Vai trò\n! rowspan=\"2\" |Vai diễn\n! rowspan=\"2\" |Ghi chú\n|-\n! Đạo diễn\n! Sản xuất\n|-\n|2001\n|''Hương dẻ''\n| {{Không}}\n| {{Không}}\n|Na lớn\n|rowspan=\"2\" |phim truyền hình\n|-\n|2004\n|''Rouge''\n| {{Không}}\n| {{Không}}\n|Thủy\n|-\n| rowspan=\"3\" | 2006\n|''Ngôi nhà bí ẩn''\n| {{Không}}\n| {{Không}}\n|Đạo diễn Trúc\n|\n|-\n|''2 trong 1''\n| {{Không}}\n| {{Không}}\n|Như Lan\n| \n|-\n|''Chuyện tình Sài Gòn''\n| {{Không}}\n| {{Không}}\n|Tâm\n|2008 chiếu ở Việt Nam\n|-\n|2007\n|''[[Dòng máu anh hùng]]''\n| {{Không}}\n| {{Không}}\n|Võ Thanh Thúy\n|\n|-\n|2009\n|''[[Bẫy rồng]]''\n| {{Không}}\n| {{Không}}\n|Trinh / Phượng hoàng\n|\n|-\n|2010\n|''Biệt đội ưng biển''\n| {{Không}}\n| {{Không}}\n|Nữ cảnh sát đa quốc gia Tú Mi\n|\n|-\n|2011\n|''Khao khát đỉnh cao''\n| {{Không}}\n| {{Có}}\n|Ngô Thanh Vân\n|phim tài liệu của [[365daband|365]] (nhóm nhạc do cô thành lập)\n|-\n| rowspan=\"3\" |2012\n|''Ngọc viễn đông''\n| {{Không}}\n| {{Không}}\n|Huyên\n|\n|-\n|''[[Ngôi nhà trong hẻm]]''\n| {{Không}}\n| {{Không}}\n|Thảo\n|-\n|''Hành trình''\n| {{Không}}\n| {{Không}}\n|Người chị\n|Phim ngắn\n|-\n|2013\n|''[[Lửa Phật]]''\n| {{Không}}\n| {{Không}}\n|Ánh\n|\n|-\n|2015\n|''Ngày nảy ngày nay''\n| {{Không}}\n| {{Có}}\n|Đan Nương\n|\n|-\n| rowspan=\"3\" |2016\n|''[[Siêu trộm (phim)|Siêu trộm]]''\n| {{Không}}\n| {{Không}}\n|Kỳ\n|Vai diễn khách mời\n|-\n|''Ngọa hổ tàng long 2''\n| {{Không}}\n| {{Không}}\n|Mantis\n|\n|-\n|''[[Tấm Cám: Chuyện chưa kể]]''\n| {{Có}}\n| {{Có}}\n|Dì ghẻ\n|\n|-\n| rowspan=\"4\" |2017\n|''[[Blade & Soul|Blade & Soul: Sứ mệnh người được chọn]]''\n| {{Không}}\n| {{Không}}\n|Sát thủ\n|Phim ngắn\n|-\n|''[[Cô Ba Sài Gòn]]''\n| {{Không}}\n| {{Có}}\n|Bà Thanh Mai\n|\n|-\n|''[[Star Wars: Jedi cuối cùng]]''\n| {{Không}}\n| {{Không}}\n|Paige Tico\n|\n|-\n|''[[Chiếc đũa quyền năng]]''\n| {{Không}}\n| {{Không}}\n|Tien\n|\n|-\n| rowspan=\"2\" |2018\n|''[[Về quê ăn Tết]]''\n| {{Không}}\n| {{Có}}\n|Đậu Xanh\n|\n|-\n|''[[Song lang (phim)|Song lang]]''\n| {{Không}}\n| {{Có}}\n|\n|\n|-\n|2019\n|''[[Hai Phượng]]''\n| {{Không}}\n| {{Có}}\n|Hai Phượng\n|\n|-\n| rowspan=\"3\" |2020\n|''Chim ưng đen''\n| {{Không}}\n| {{Có}}\n| Điệp vụ Hương\n| Phim ngắn\n|-\n|''[[Năm chiến hữu]]''\n| {{Không}}\n| {{Không}}\n|Hanoi Hannah\n|\n|-\n|''[[The Old Guard: Những chiến binh bất tử]]''\n| {{Không}}\n| {{Không}}\n|Quỳnh\n|\n|-\n|2021\n|''[[Trạng Tí phiêu lưu ký]]''\n| {{Không}}\n| {{Có}}\n|\n|\n|-\n| rowspan=\"2\" |2022\n|''The Princess''\n| {{không}}\n| {{không}}\n| Linh\n|\n|-\n|''[[Thanh Sói – Cúc dại trong đêm]]''\n| {{Có}}\n| {{Có}}\n|Dì Lin \"Jacqueline\"\n|\n|-\n|2023\n| ''[[Kẻ kiến tạo]]''\n| {{Không}}\n| {{Không}}\n| Kami\n|\n|-\n|2025\n| ''The Old Guard 2''\n| {{Không}}\n| {{Không}}\n| Quỳnh\n|\n|-\n|}"
            },
            {
                "title": "Chương trình truyền hình",
                "subsection_wikitext": "* 2009: ''[[Hành trình kết nối những trái tim]]''\n* 2010: ''[[Bước nhảy hoàn vũ (2010)|Bước nhảy hoàn vũ]]''\n* 2012, 2013: ''[[Thử thách cùng bước nhảy (chương trình truyền hình)|Thử thách cùng bước nhảy]]'' (giám khảo)\n* 2013: ''[[Project Runway Vietnam: Nhà thiết kế Thời trang Việt Nam|Project Runway Vietnam]]'' (dẫn chương trình)"
            },
            {
                "title": "Diễn xuất trong MV",
                "subsection_wikitext": "* Awakening ([[365daband|365]])\n* Ác mộng ([[365daband|365]])\n* Chia xa ([[Tuấn Hưng]])\n* Dạ khúc cho tình nhân ([[Đàm Vĩnh Hưng]])\n* Dẫu tình đã xa ([[Minh Thuận]]) \n* Hai cô tiên ([[365daband|365]]) (cảnh trong phim ''Ngày nảy ngày nay'')\n* Hai cô tiên (Dance version) ([[365daband|365]])\n* Merry Christmas ([[365daband|365]])\n* Mưa (Avi Kim Anh)\n* Ngọc Lan ([[Trần Thu Hà]])\n* Về quê ăn Tết ([[Jun Phạm]])"
            }
        ]
    
    '''
    
    all_films_section = extract_films_section(wikitext)
    
    films_from_subsection_dict = extract_films_from_subsection(all_films_section)
    films_by_type = {}  # dict kết quả cuối cùng

    for subsection in films_from_subsection_dict:
        type_name = subsection["title"]  # ví dụ: "Phim điện ảnh"
        sub_wikitext = subsection["subsection_wikitext"].strip()

        # --- Nếu không phải bảng wikitable thì bỏ qua ---
        if not sub_wikitext.startswith("{|"):
            print(f"Bỏ qua '{type_name}' (không phải bảng wikitable)")
            continue

        # --- Xử lý bảng wikitable ---
        headers = extract_headers(sub_wikitext)
        df_films = wikitable_to_df(sub_wikitext, headers)
        films_dict = df_to_filted_vn_film_dict(df_films)

        # --- Ghi vào dict kết quả ---
        films_by_type[type_name] = films_dict
        

    return films_by_type

#test 
print_json_format(extract_films_from_person_wikitext(get_wikitext("Trấn Thành")))
# print_json_format(extract_films_from_person_wikitext(ndln_wikitext))

# %% [markdown]
# ## get_all_films_from_films_dict

# %%
def get_all_films_from_films_dict(films_dict):
    all_films = []
    for type_name, films in films_dict.items():
        for film in films:
            # thêm thông tin loại phim để dễ phân biệt
            film_with_type = film.copy()
            film_with_type["Type_film"] = type_name
            all_films.append(film_with_type)
    return all_films

#test
k0 = extract_films_from_person_wikitext(get_wikitext("Trấn Thành"))
k1 = get_all_films_from_films_dict(k0)

k0 = extract_films_from_person_wikitext(ndln_wikitext)
k1 = get_all_films_from_films_dict(k0)

for f in k1:
    print(f)

# %% [markdown]
# ## extract_cast_section

# %%
# def extract_cast_section(wikitext):
#     text_lower = wikitext.lower()
    
#     # == dien vien == , ==dien vien== 
#     keywords = [
#         "== diễn viên ==",
#         "==diễn viên==",
#     ]

#     # tìm vị trí tiêu đề '== diễn viên =='
#     start_position = -1
#     for k in keywords: 
#         pos = text_lower.find(k)
#         if pos != -1:
#             start_position = pos
#             key = k
#             break

    
    
#     if start_position == -1: return "" # str rỗng, ko có 

#     # cắt phần vb sau title dvien
#     content = wikitext[start_position + len(key):] 

#     # tìm vị trí tiêu đề tiếp theo (== ... ==)
#     end_position = content.find("==")
#     if end_position != -1: section = content[:end_position] 
#     else: section = content # k có title next -> lay het 

#     return section.strip() # delete space thừa


# print(extract_cast_section(matBiec_wikitext))

# %%
#test

nhaBaNu = get_wikitext("Nhà bà Nữ")

# %% [markdown]
# ## extract_cast_list

# %%
# import re
# def extract_cast_list(wikitext):
#     wikicode = mwparserfromhell.parse(wikitext)
#     cast_list = []

#     # lay all dòng begin * :*[[Trúc Anh]]
#     lines = []
#     # duyệt từng node => chuyển node thành chuỗi, bỏ khoảng trắng thừa => if line begin * -> add vào lines

#     # default 
#     actor_name_list = []
#     character_list = []
    
#     for node in wikicode.nodes:  
#         # bỏ qa mọi dạng template {{...}}
#         if node.__class__.__name__ == "Template": continue

#         text = str(node).strip()  

#         if ("*" in text) or (not text):
#             continue
        
#         # lấy những line begin [[ , vai 
#         # [[Trúc
#         # vai Hà Lan
#         if text.startswith('[[Tập tin:'): continue       
#         if text.startswith('[['):       
#             if ("[[" in text) and ("]]" in text):
#                 text = text.replace("[[", "").replace("]]", "")
#                 if "|" in text:
#                     text = text.split("|")[0].strip()      
#             actor_name_list.append(text)

#         if (text.startswith('vai')) or (text.startswith('trong vai')):
          
#             match = re.search(r"vai\s+([^,.:]+)", text)
          
#             if match: 
#                 text = match.group(1).strip()
#                 # if res begin "trong", bỏ trong
#                 if text.startswith("trong "):
#                     text = text[6:].strip()

#                 character_list.append(text)
#             else: print("Không tìm thấy vai")


#     for actor, character in zip(actor_name_list, character_list):
#         actor_obj = {
#             "actor_name": actor,
#             "character": character
#         }
#         # add list
#         cast_list.append(actor_obj)

    
#     return cast_list


# #test 

# print(extract_cast_list(extract_cast_section(matBiec_wikitext)))
# print(len(extract_cast_list(extract_cast_section(matBiec_wikitext))))



# %%
# #test 

# print(extract_cast_list(extract_cast_section(nhaBaNu)))

# %% [markdown]
# [[Tập tin:Mắt biếc – Behind the scenes (2).png|nhỏ|Dàn diễn viên phim ''Mắt biếc'']] 
#  
# * 
# [[Trần Nghĩa (diễn viên)|Trần Nghĩa]] 
# vai Ngạn: 

# %% [markdown]
# ## extract_actors_from_film

# %%
# def extract_actors_from_film(wikitext):
#     # Trích danh sách diễn viên từ '== Diễn viên ==' của wikitext phim
#     cast_section = extract_cast_section(wikitext)
    
#     #list 
#     cast_list = extract_cast_list(cast_section)
#     # {name: , char: }


#     vn_cast_list = []
#     for cast in cast_list:
#         actor_name = cast.get('actor_name')  # lấy tên diễn viên
#         if not actor_name: continue  # bỏ qua 

#         if is_vn_person_wikilink(actor_name):
#             vn_cast_list.append(cast)
        
#     return vn_cast_list

# #test
# matBiec_actors = extract_actors_from_film(matBiec_wikitext)

# print(matBiec_actors)

# %% [markdown]
# ## extract_directors_from_film

# %%
def extract_directors_from_film(wikitext):
    infobox = extract_infobox_from_wikitext(wikitext)
    # Đưa tất cả key về lowercase 
    infobox = {k.lower(): v for k, v in infobox.items()}

    possible_keys = ['đạo diễn', 'director', 'directors']
    director = None
    for key in possible_keys:
        if key in infobox:
            director = infobox[key]
            break
        
    if not director:
        return None

    # Parse wikicode từ giá trị của trường đạo diễn
    wikicode = mwparserfromhell.parse(director)
    directors = []

    for link in wikicode.filter_wikilinks():
        # Lấy phần title trong [[title|...]]
        title = str(link.title).strip()
        if title:
            directors.append(title)

    # Nếu không có wikilink => bỏ qua
    if not directors:
        return "đạo diễn không có wikilink"

    return directors if len(directors) > 1 else directors[0]

#test
print(extract_directors_from_film(matBiec_wikitext))

# %%
SEED_LIST = read_seeds(SEED_LIST_FILE)

print(SEED_LIST)

# %%
# print(extract_actors_from_film(get_wikitext("Dòng máu anh hùng")))

# %%
# print(extract_actors_from_film(get_wikitext("Bộ tứ báo thủ")))

# %%
def clean_infobox_markup(text):
    """
    Làm sạch wiki markup trong infobox:
    Giữ lại text hiển thị, bỏ [[link]], {{template}}, thẻ HTML, ký hiệu...
    """
    if not text:
        return ""

    code = mwparserfromhell.parse(text)

    # Wikilinks [[abc]] , [[abc|xyz]]
    for link in code.filter_wikilinks():
        display = str(link.text or link.title)
        text = text.replace(str(link), display)

    # Templates {{...}} 
    for tpl in code.filter_templates():
        name = tpl.name.strip().lower()

        # {{VIE}} or {{vi}} → Việt Nam
        if name in ("vie", "vi", "vietnam"):
            text = text.replace(str(tpl), "Việt Nam")

        # {{birth date and age|1995|06|04}} → 1995-06-04
        elif name.startswith("birth date"):
            try:
                y, m, d = [tpl.get(i).value.strip() for i in range(1, 4)]
                text = text.replace(str(tpl), f"{y}-{m.zfill(2)}-{d.zfill(2)}")
            except Exception:
                text = text.replace(str(tpl), "")

        # {{flatlist|...}}, {{plainlist|...}}, {{hlist|...}}
        elif name in ("flatlist", "plainlist", "hlist", "ubl"):
            try:
                inner = str(tpl.get(1)).replace("\n", " ").replace("*", " ")
                text = text.replace(str(tpl), inner.strip())
            except Exception:
                text = text.replace(str(tpl), "")

        # {{flag|VIE}} → Việt Nam
        elif name == "flag":
            try:
                text = text.replace(str(tpl), str(tpl.get(1)).strip())
            except Exception:
                text = text.replace(str(tpl), "")

        else:
            text = text.replace(str(tpl), "")

    # Xóa thẻ HTML và ký hiệu 
    text = re.sub(r"<ref[^>]*>.*?</ref>", "", text, flags=re.DOTALL)
    text = re.sub(r"<!--.*?-->", "", text, flags=re.DOTALL)  # comment
    text = re.sub(r"<br\s*/?>", ", ", text)  # xuống dòng -> dấu phẩy
    text = re.sub(r"<[^>]+>", "", text)  # thẻ còn lại
    text = re.sub(r"'{2,}", "", text)  # '' hoặc ''' in nghiêng/đậm
    text = re.sub(r"&[a-z]+;", " ", text)  # ký hiệu HTML

    # Làm gọn lại khoảng trắng
    text = re.sub(r"\s+", " ", text).strip()

    return text


print(clean_infobox_markup("[[Ngô Thanh Vân]] là [[đạo diễn]]"))
print(clean_infobox_markup("[[Phim điện ảnh|Phim]] do [[Ngô Thanh Vân]] đạo diễn"))
print(clean_infobox_markup("[[A]] và [[B]] là [[C]]"))
print(clean_infobox_markup('[[Đại học Assumption (Thái Lan)|Đại học Assumption]]'))
print(clean_infobox_markup('[[Hà Nội]], [[Việt Nam]]'))
print(clean_infobox_markup('[[Hà Nội]], [[Việt Nam]]'))




# %% [markdown]
# ## test

# %%
#test
z1 = "Ngô Thanh Vân"
z2 = extract_films_from_person_wikitext(get_wikitext(z1))
print(z1,get_all_films_from_films_dict(z2))

print('=========================================================')

z1 = "Lý Hải"
z2 = extract_films_from_person_wikitext(get_wikitext(z1))
print(z1,get_all_films_from_films_dict(z2))

print('=========================================================')

z1 ="Victor Vũ"
z2 = extract_films_from_person_wikitext(get_wikitext(z1))
print(z1,get_all_films_from_films_dict(z2))
print( '=========================================================')

z1 = "Trấn Thành"
z2 = extract_films_from_person_wikitext(get_wikitext(z1))
print(z1,get_all_films_from_films_dict(z2))

print('=========================================================')

z1 = "Charlie Nguyễn"
z2 = extract_films_from_person_wikitext(get_wikitext(z1))
print(z1,get_all_films_from_films_dict(z2))

print('=========================================================')

z1 = "Jun Vũ"
z2 = extract_films_from_person_wikitext(get_wikitext(z1))
print(z1,get_all_films_from_films_dict(z2))

# %%
def clean_infobox_wikidict(data):
    cleaned_dict = {}
    for key in data:
        value = data[key]
        cleaned_value = clean_infobox_markup(value)
        cleaned_dict[key] = cleaned_value
    return cleaned_dict

#test 

h = "Ninh Dương Lan Ngọc"
h0 = get_wikitext(h)
h1 = extract_infobox_from_wikitext(h0) 
print(h1)
h2 = clean_infobox_wikidict(h1)
print(h2)

# %% [markdown]
# ## get actor

# %%
def is_film_infobox(infobox_dict):
    # neu la infobox của actor, director => ko là infobox film => False 
    if is_actor(infobox_dict) or is_director(infobox_dict): 
        print('=>is_film_infobox: là infobox actor/director, ko là film')
        return False
    keyword = "phim"
    for k, v in infobox_dict.items():
        if keyword in str(k).lower() or keyword in str(v).lower():
            return True
    return False

#test
print('ndln','-->',is_film_infobox(extract_infobox_from_wikitext(ndln_wikitext)))

c = "Mùi ngò gai"
c1 = get_wikitext(c)
c2 = extract_infobox_from_wikitext(c1)
c3 = is_film_infobox(c2)
print(c,'-->',c3)

c = "Mắt biếc (phim)"
c1 = get_wikitext(c)
c2 = extract_infobox_from_wikitext(c1)
c3 = is_film_infobox(c2)
print(c,'-->',c3)

 
c = "Gia sư nữ quái"
c1 = get_wikitext(c)
c2 = extract_infobox_from_wikitext(c1)
c3 = is_film_infobox(c2)
print(c,'-->',c3)

p1 = "Cổng mặt trời (phim truyền hình)"
c = p1
c1 = get_wikitext(c)
c2 = extract_infobox_from_wikitext(c1)
c3 = is_film_infobox(c2)
print(c,'-->',c3)

# %%
# extract_films_section from wiki person 

def extract_actors_section(wikitext,depth=0):
  

    # check là actor or director 
    infobox_dict = extract_infobox_from_wikitext(wikitext)
    
    if depth == 0 and not is_film_infobox(infobox_dict): 
        return "==> cần extract actor mà wikitext ko là film"
    
    """
    Tách phần wikitext thuộc tiêu đề cấp 2 (== ... ==)
    có 'diễn viên' hoặc 'actor' trong tiêu đề.
    Giữ nguyên wikitext gốc, chỉ so khớp tên tiêu đề ở dạng thường.
    Nếu section chỉ chứa liên kết {{chính|...}} thì đệ quy sang trang đó.
    """
    if not wikitext or depth > 2: return ""

    lines = wikitext.splitlines()

    headings = []  # lưu (index_dòng, heading_dict)

    for i, line in enumerate(lines):
        h = parse_heading(line)
        if h:
            headings.append((i, h))

    # chọn heading cấp 2 có từ "phim"/"film"
    chosen = None
    for i, h in headings:
        title_lower = h['title'].lower()
        if h['level'] == 2 and ('diễn viên' in title_lower 
                                or 'actor' in title_lower 
                                or 'actors' in title_lower 
                                or 'nhân vật' in title_lower
                                or 'character' in title_lower):
            chosen = (i, h)
            break
        

    if not chosen: return ""

    start_line_index = chosen[0]
    chosen_level = chosen[1]['level'] # 2 => level 2 

    # tìm heading cấp 2 kế tiếp để biết điểm kết thúc
    end_line_index = len(lines)
    for i, h in headings:
        if i > start_line_index and h['level'] == chosen_level:
            end_line_index = i
            break

    # trích phần nội dung giữa hai heading
    section = "\n".join(lines[start_line_index + 1:end_line_index]).strip()
    
    # --- kiểm tra nếu chỉ là liên kết {{chính|...}} hoặc {{main|...}} ---
    main_re = re.search(r"\{\{\s*(?:chính|main)\s*\|\s*([^}|]+)", section, flags=re.IGNORECASE)
    if main_re:
        linked_title = main_re.group(1).strip()
        try:
            linked_wikitext = get_wikitext(linked_title)
            if linked_wikitext:
                # đệ quy sang trang phụ, tăng depth
                return extract_actors_section(linked_wikitext, depth=depth + 1)
        except Exception as e:
            print("Lỗi khi lấy wikitext:", e)
    return section

# test

# print('================================')
# print(extract_actors_section(get_wikitext("Charlie Nguyễn")))
# print('================================')
# print(extract_actors_section(get_wikitext("Mùi ngò gai")))
p1 = "Chuyện tình mùa thu"
print(extract_actors_section(get_wikitext(p1)))

# %%
def extract_table_rows(wikitext):
    wikitext = wikitext.replace('|-', "@@@")
    parts = wikitext.split('@@@')
    return [p.strip() for p in parts[1:] if p.strip()]

#test 

rows = extract_table_rows(extract_actors_section(get_wikitext("Mùi ngò gai")))
for i, r in enumerate(rows, 1):
    r = r.lower()
    if "lương" in r:
        print(f"--- Row {i} ---")
        print(r)




    

# %%
def clean_string(s):
    return s.replace("'", "").strip()

text = "''Sanaly''\n"
result = clean_string(text)
print(result) 

def extract_all_wikilink(s):
    matches = re.findall(r'\[\[.*?\]\]', s)
    return matches

text = '==  style="text-align:center;"  - colspan="2" [[Lương Thế Thành]]<ref name=":2" [[ninh dương lan ngọc]]/>'
result = extract_all_wikilink(text)
print(result) 

# %% [markdown]
# ### extract_actors_and_roles_from_wikitable_row

# %%
def extract_actors_and_roles_from_wikitable_row(row_text):
    """
    Nhận 1 đoạn văn bản dạng wiki table row (ví dụ: |''Vy''|[[Angela...]]|[[Hồng Ánh]] ...)
    -> Trả về list các dict: {character: ..., actor_name: ...}
    """
    row_text = row_text.strip("|")

    # Tách theo | nhưng KHÔNG nằm trong [[...]]
    parts = re.split(r'\|(?![^[]*\]\])', row_text)
   
    if not parts:
        print('=> no part in row text')
        return []

    # Cột đầu là vai diễn
    character = ""
    for p in parts:
        if "''" in p:
            # Lấy nội dung trong ''...'' nếu có
            character = p.replace("''", "").strip()
            break

    results = []
   
    string = ""
    for p in parts[1:]:
        string += p 

    actor_wikilinks = extract_all_wikilink(string)
    for link in actor_wikilinks:
        actor_name =clean_wiki_markup(link)
        if actor_name:
                results.append({
                    "actor_name": actor_name,
                    "character": character
                    
                })
    # lọc cast vn
    res_filtered = [obj for obj in results if is_vn_person_wikilink(obj["actor_name"])]

    return res_filtered



# --- ví dụ ---
text = """|''Vy''
|[[Angela Phương Trinh]]/[[Phạm Thị Ngọc Trinh|Ngọc Trinh]] (†)<ref name=":2" />
|Ngọc Trinh(†)
|[[Hồng Ánh]]<ref name=":2" />"""
print(extract_actors_and_roles_from_wikitable_row(text))
text = '|\'\'Vy\'\'\n|[[Angela Phương Trinh]]/[[Phạm Thị Ngọc Trinh|Ngọc Trinh]] (†)<ref name=":2" />\n|Ngọc Trinh(†)\n|[[Hồng Ánh]]<ref name=":2" />'
print(extract_actors_and_roles_from_wikitable_row(text))
text = """|''thiện''
| style="text-align:center;" | -
| colspan="2" |[[Lương Thế Thành]]<ref name=":2" />

"""

print(extract_actors_and_roles_from_wikitable_row(text))

# %% [markdown]
# ### extract_actors_from_wikitable

# %%
def extract_actors_from_wikitable(section_wikitext):
    
    wikitable_rows = extract_table_rows(section_wikitext)  # dạng |- abc ... |- 

    actors_list = []
    for r in wikitable_rows:
        obj = extract_actors_and_roles_from_wikitable_row(r)
        if obj: actors_list.append(obj)

    return actors_list


print(len(extract_actors_from_wikitable(extract_actors_section(get_wikitext('Mùi ngò gai')))))
print(extract_actors_from_wikitable(extract_actors_section(get_wikitext('Mùi ngò gai'))))

# %%
m = 'Kim Hiền (diễn viên)'
n = get_wikitext(m)
tenfile = "kimhien"
# save_to_txt(n,tenfile)

info = extract_infobox_from_wikitext(n)
print(info)

# %%
import re
def is_vn_person_wikilink(name):
    # Kiem tra nhanh ten hop le cua dien vien
    # Tra ve True neu co kha nang la ten nguoi Viet/ten hop le
    # Loai bo cac chuoi nhu 'png', 'nhỏ', 'dàn', 'phim', 'gồm' o dau chuoi
    if not name or len(name) < 2 or len(name) > 80:
        return False
    bad_prefixes = [r'^png', r'^nhỏ', r'^dan', r'^phim', r'^gom', r'^co ', r'^có ']
    for p in bad_prefixes:
        if re.search(p, name, flags=re.IGNORECASE):
            return False
    # Neu chuoi chi toan so hoac ky tu la vo nghia thi loai
    if re.fullmatch(r'[\W_]+', name):
        return False
    # Co it nhat mot chu cai
    if not re.search(r'[A-Za-z\u00C0-\u017F]', name):
        return False
    return True

def extract_actors_from_text(wikitext):
    # parse wiki va chuyen thanh chuoi
    wikicode = mwparserfromhell.parse(wikitext)
    raw = str(wikicode)

    # 1 Lam sach toan van: loai bo tag ref va chuan hoa khoang trang
    raw = re.sub(r"<ref[^>]*>.*?</ref>", "", raw, flags=re.DOTALL)
    raw = re.sub(r"<ref[^/]*/>", "", raw)
    raw = " ".join(raw.split())

    # 2 Tach tung muc theo dau '*'
    items = re.split(r"\*", raw)
    cast_list = []

    for item in items:
        item = item.strip()
        if not item:
            continue

        # Loai bo cac prefix du thua nhu 'phim gom', 'co su tham gia', 'png|nho|...'
        item = re.sub(
            r"^(?:[^A-ZÀ-Ỵ]*\b(?:gồm|co|có|voi|với|trong|png|nhỏ|Dan|Dàn|phim)[^A-ZÀ-Ỵ]*)",
            "",
            item,
            flags=re.IGNORECASE
        )

        # 4 Lay actor
        # Neu co link [[...|...]] hoac [[...]] thi lay phan truoc dau '|'
        m_link = re.search(r"\[\[(.+?)(?:\|([^\]]+))?\]\]", item)
        if m_link:
            actor = m_link.group(1)
            # vi tri bat dau va ket thuc cua match de cat phan con lai
            start_pos = m_link.start()
            end_pos = m_link.end()
        else:
            # Neu khong co link thi lay phan truoc 'vai' hoac 'trong vai'
            parts = re.split(r"(?:trong\s+vai|vai)\s+", item, maxsplit=1, flags=re.I)
            if len(parts) > 1:
                actor = parts[0].strip()
                # tim vi tri actor trong chuoi de cat tail
                match_actor = re.search(re.escape(actor), item)
                start_pos = match_actor.start() if match_actor else 0
                end_pos = match_actor.end() if match_actor else 0
            else:
                actor = ""
                start_pos = end_pos = 0

        # 5 Lam sach ten actor: bo cac phan thua o dau, bo ky tu dac biet, chuan hoa space
        actor = re.sub(r"^(?:.*?\*|.*?\b(?:gồm|gom|cung|cùng|voi|với)\b)\s*", "", actor, flags=re.IGNORECASE)
        actor = re.sub(r"^(?:va|và|,)\s*", "", actor, flags=re.IGNORECASE)

        # Xoa cac dau cau nhung GIU nguyen chu co dau
        actor = re.sub(r"[.,;:!?\"“”‘’()\[\]{}<>|\\/]", "", actor)
        # Bo nhieu khoang trang thanh 1
        actor = " ".join(actor.split())

        if not actor:
            continue

        # 3 Lay role chi trong phan sau actor de tranh an nham role cua muc truoc
        tail = item[end_pos:]
        m_role = re.search(r"(?:trong\s+vai|vai)\s+(.+?)(?:[,.:]|$)", tail, flags=re.I)
        role = m_role.group(1).strip(" '\"") if m_role else ""
        # Lam sach role: chuyen ve thuong, loai bo cau muc khong can thiet
        
      
        role = re.sub(r"[\'\"]", "", role).strip()
        role = " ".join(role.split())
        role = role.replace("Cùng một số diễn viên khác",'').strip()

        if not actor or not role:
            continue

        cast_list.append({
            "actor_name": actor,
            "character": role
        })

    # Loc ket qua: chi giu nhung doi tuong co ve la ten dien vien hop le
    res_filtered = [obj for obj in cast_list if is_vn_person_wikilink(obj["actor_name"])]
    return res_filtered
#test 

print(extract_actors_from_text(extract_actors_section(matBiec_wikitext)))
print(len(extract_actors_from_text(extract_actors_section(matBiec_wikitext))))

'''
*
[[Trần Nghĩa (diễn viên)|Trần Nghĩa]]
 vai Ngạn:
<ref>{{Chú thích web|url=http://baodatviet.vn/van-hoa/tin-tuc-giai-tri/victor-vu-truy-lung-nang-mat-biec-va-chang-si-tinh-3368589/|title=Victor Vũ truy lùng nàng Mắt Biếc và chàng si tình|last=|first=|date=5 tháng 11 năm 2018|website=Báo Đất Việt|archive-url=https://web.archive.org/web/20190716065358/http://baodatviet.vn/van-hoa/tin-tuc-giai-tri/victor-vu-truy-lung-nang-mat-biec-va-chang-si-tinh-3368589/|archive-date=2019-07-16|url-status=dead|access-date=20 tháng 1 năm 2019}}</ref>
<ref>{{chú thích web|url=https://ione.vnexpress.net/tin-tuc/phim/vietnam/vuot-qua-hon-1-400-ung-vien-mat-biec-cua-victor-vu-da-lo-dien-3886413.html|website=IONE.VNEXPRESS.NET|tiêu đề=Vượt qua hơn 1.400 ứng viên, 'Mắt Biếc' của Victor Vũ đã lộ diện|ngày truy cập=2019-02-27|ngày=2019-02-26|ngôn ngữ=vi}}</ref>
 Chàng trai sinh ra và lớn lên ở làng Đo Đo, mang dáng vẻ thư sinh cùng tâm hồn nghệ sĩ. Ngạn yêu nhất hai điều trong đời mình: ngôi làng Đo Đo và Hà Lan. Tình yêu Ngạn dành cho Hà Lan gần như vĩnh cửu, không hề phai nhòa sau 30 năm dù Hà Lan đã đem lòng yêu Dũng và hình bóng của Dũng. Phạm Đình Thái Ngân lồng tiếng cho phần hát của nhân vật.
<ref>{{chú thích web|url=https://vnexpress.net/giai-tri/thai-ngan-toi-khong-luy-tinh-nhu-ngan-mat-biec-4032451.html|tiêu đề=Thái Ngân: 'Tôi không lụy tình như Ngạn Mắt biếc'|website=Báo điện tử [[VnExpress]]|ngôn ngữ=vi|url lưu trữ=https://web.archive.org/web/20191226235344/https://vnexpress.net/giai-tri/thai-ngan-toi-khong-luy-tinh-nhu-ngan-mat-biec-4032451.html|ngày lưu trữ=2019-12-26|ngày truy cập=2020-01-01|url-status=dead}}</ref>


*
'''
p1 = "Chuyện tình mùa thu"
print((extract_actors_from_text(extract_actors_section(get_wikitext(p1)))))

# %% [markdown]
# ### extract_actors_from_list

# %%
def extract_actors_from_list(wikitext):
    wikicode = mwparserfromhell.parse(wikitext)
    result = []
    for node in wikicode.nodes:     
        text = str(node).strip() 
        
     
        if text.startswith('{{cast list|'): 
   

            parts = text.split('*')

            for line in parts:
                line = line.lstrip('*').strip()
                if not line:
                    continue

                # Xóa <ref>...</ref>, <ref .../>, {{...}}
                line = re.sub(r"<ref[^>]*>.*?</ref>", "", line)
                line = re.sub(r"<ref[^>]*\/>", "", line)
                line = re.sub(r"\{\{[^\}]+\}\}", "", line)

                # Regex cho tên diễn viên
                actor_pattern_linked = r"\[\[([^\]|]+)(?:\|[^\]]+)?\]\]"
                actor_pattern_plain = r"^([^\[]\S.+?)\s+(?:trong|vào)?\s*vai"

                # Regex cho vai diễn
                char_pattern = r"(?:trong|vào)?\s*vai\s*''([^']+)''"

                actor_match = re.search(actor_pattern_linked, line)
                if actor_match:
                    actor_name = actor_match.group(1).strip()
                else:
                    actor_match_plain = re.search(actor_pattern_plain, line)
                    actor_name = actor_match_plain.group(1).strip() if actor_match_plain else None

                char_match = re.search(char_pattern, line)
                character = char_match.group(1).strip() if char_match else None

                if actor_name or character:
                    result.append({'actor_name': actor_name, 'character': character})
    result_filtered = [obj for obj in result if is_vn_person_wikilink(obj["actor_name"])]
    return result_filtered
    

                
        
    # # Tìm tất cả các dòng bắt đầu bằng '*'
    # lines = [str(node).strip() for node in wikicode.nodes if str(node).strip().startswith('*')]

    # result = []
    # for line in lines:
    #     line = line.lstrip('*').strip()

    #     print(line)

    # return result


p1 = "Cổng mặt trời (phim truyền hình)"
print(extract_actors_from_list((extract_actors_section(get_wikitext(p1)))))

# %%
#chung
def is_wikitable(text):
    code = mwparserfromhell.parse(text)
    tables = code.filter_tags(matches=lambda node: node.tag == "table")
    wikitable_syntax = any(str(node).startswith("{|") for node in code.nodes)
    return bool(tables) or wikitable_syntax

#test
#chung
def is_list(text):
    code = mwparserfromhell.parse(text)
    tables = code.filter_tags(matches=lambda node: node.tag == "list")
    wikitable_syntax = any(str(node).startswith("{|") for node in code.nodes)
    # Template kiểu {{cast list|...}}, {{Plainlist|...}}, {{ubl|...}}, v.v.
    templates = code.filter_templates(matches=lambda t: "list" in t.name.strip().lower() or t.name.strip().lower() in {"ubl", "unbulleted list", "plainlist"})
    return bool(tables or wikitable_syntax or templates)

print(is_wikitable(extract_actors_section(get_wikitext("Mùi ngò gai"))))
p1 = "Cổng mặt trời (phim truyền hình)"
is_list(extract_actors_section(get_wikitext(p1)))

# %%
def extract_actors_from_film_wikitext(film_wikitext):
    
    section_wikitext = extract_actors_section(film_wikitext)
    # print(repr(section_wikitext[:300]))
    # print(is_wikitable(section_wikitext))
    res = []
    if is_wikitable(section_wikitext):
        actors = extract_actors_from_wikitable(section_wikitext)
        if actors: res.extend(actors) # có r thì nối list 
    elif is_list(section_wikitext):
        actors = extract_actors_from_list(section_wikitext)
        if actors: res.extend(actors) # có r thì nối list 
    else:
        actors = extract_actors_from_text(section_wikitext)
        if actors: res.extend(actors)
    
    return res

#test

print('mùi ngò gai',extract_actors_from_film_wikitext((get_wikitext("Mùi ngò gai"))))
print('mắt biếc',extract_actors_from_film_wikitext((get_wikitext("Mắt biếc (phim)"))))


# %% [markdown]
# ## test 1

# %%
# test director của mùi ngò gai ? 

# print(extract_directors_from_film(get_wikitext("Mùi ngò gai"))) # đạo diễn không có wikilink
t1 = "Chuyện tình mùa thu"
# print(extract_directors_from_film(get_wikitext(t1))) #đạo diễn k có wikilink



# %%

p1 = "Lựa chọn số phận"
print(p1,extract_actors_from_film_wikitext((get_wikitext(p1))))




['Trấn Thành', 'Ninh Dương Lan Ngọc', 'Ngô Thanh Vân', 'Hồng Đào', 'Kiều Minh Tuấn', 'Victor Vũ', 'Charlie Nguyễn', 'Vũ Ngọc Đãng', 'Kaity Nguyễn', 'Jun Vũ', 'Mạnh Trường', 'Phương Oanh']
{{for|tiểu thuyết cùng tên|Mắt biếc (tiểu thuyết)}}
{{Thông tin phim
| tên = Mắt biếc
| hình = Áp phích phim Mắt biếc.jpg
| caption = Áp phích chiếu rạp của phim tại Việt Nam
| đạo diễn = [[Victor Vũ]]
| sản xuất = {{Liệt kê|
* Nguyễn Quốc Công
* Kay Nguyễn
}}
| kịch bản = {{Liệt kê|
* Victor Vũ
* A Type Machine
}}
| dựa trên = ''[[Mắt biếc (tiểu thuyết)|Mắt biếc]]'' của [[Nguyễn Nhật Ánh]]
| diễn viên = {{Liệt kê|
* [[Trần Nghĩa (diễn viên)|Trần Nghĩa]]
* [[Trúc Anh]]
* Trần Phong
* Khánh Vân
* Thảo Tâm
}}
| âm nhạc = [[Christopher Wong]]<br>[[Phan Mạnh Quỳnh]]
| quay phim = [[Dominic Pereira]]<ref>{{chú thích web|url=https://danviet.vn/nhung-guong-mat-nuoc-ngoai-thanh-cong-voi-dien-anh-viet-20220916084332568.htm|title=Những gương mặt nước ngoài thành công với điện ảnh Việt|author=Thủy Vũ|date=2022-0

In [2]:
# crawl data phim và diễn viên từ danh sách seed
def crawl_data(seed_list):
    films_data = {}
    persons_data = {}
    for person in seed_list:
        wikitext = get_wikitext(person)
        print(f"Crawling data for person: {person}")
        person_info = extract_infobox_from_wikitext(wikitext)
        print(f"Person info: {person_info}")
        film_dict = extract_films_from_person_wikitext(wikitext)
        film_info = get_all_films_from_films_dict(film_dict)
        film_list = []
        for film in film_info:
            # Ưu tiên các trường phổ biến
            
            title = film.get('Tựa đề') or film.get('Tên phim')
                             
            # Nếu chưa có, thử tìm key có chứa từ "Tựa"
            if not title:
                for key in film.keys():
                    if "Tựa" in key or "Tên" in key or "Phim" in key:
                        title = film[key]
                        break  # dừng lại khi tìm thấy key đầu tiên
            if not title:
                continue

           

            film_list.append(title)
        print(f"Film info: {film_list}")
        persons_data[person] = {
            'films': film_list,
            'info': person_info
        }
        
        for film in film_list:
            if film not in films_data:
                film_wikitext = get_wikitext(film)
                film_infobox = extract_infobox_from_wikitext(film_wikitext)
                print(f"Film infobox for {film}: {film_infobox}")
                film_actors = extract_actors_from_film_wikitext(film_wikitext)
                print(f"Film actors for {film}: {film_actors}")
                film_director = extract_directors_from_film(film_wikitext)
                print(f"Film director for {film}: {film_director}")
                if (film_director == [] and film_actors == []):
                    continue  # bỏ qua phim k có dv và đđ
                films_data[film] = {
                    'infobox': film_infobox,
                    'actors': film_actors,
                    'director': film_director
                }
                
    return persons_data, films_data
persons_data, films_info = crawl_data(SEED_LIST)
print(persons_data["Mạnh Trường"])



Crawling data for person: Trấn Thành
Person info: {'tên': 'Trấn Thành', 'image': 'Trấn Thành 191226.png', 'caption': 'Trấn Thành vào năm 2019', 'tên khai sinh': 'Huỳnh Trấn Thành', 'ngày sinh': '{{ngày sinh và tuổi|1987|2|5}}', 'chiều cao': '{{convert|1,71|m}}', 'nơi sinh': '[[Thành phố Hồ Chí Minh]], [[Việt Nam]]', 'tên khác': 'A Xìn', 'công việc': '{{hlist|[[Diễn viên]]|[[Nghệ sĩ hài]]|[[Người dẫn chương trình]]|[[Doanh nhân]]|[[Nhà làm phim]]}}', 'năm hoạt động': '2006–nay', 'giải thưởng': '[[Danh sách giải thưởng và đề cử của Trấn Thành|Danh sách]]', 'quốc tịch': '{{VIE}}', 'dân tộc': '[[Người Việt|Kinh]]<br>[[Dân tộc Trung Hoa|Hoa]]', 'notable_works': '[[Sự nghiệp điện ảnh của Trấn Thành|Danh sách]]', 'quê quán': '[[Quảng Đông]], [[Trung Quốc]]', 'spouse': '{{marriage|[[Hari Won]]|2016}}', 'cha': 'Huỳnh Chấn Liêm', 'mẹ': 'Phạm Thị Hạnh Dung', 'người thân': '{{plainlist|\n* Huỳnh Trinh Mi {{small|(em gái)}}\n* [[Huỳnh Uyển Ân]] {{small|(em gái)}}\n}}', 'học vấn': '[[Trường Đại học 

In [3]:
def expand_network(person_data, film_data, max_depth=3):
    
    visited_persons = set(person_data.keys())
    current_depth = 0

    while current_depth < max_depth:
        print(f"\n Đang mở rộng mức {current_depth + 1} ...")
        new_persons = {}  

        
        for film_name, film_info in film_data.items():
            if 'actors' not in film_info:
                continue  # tránh KeyError

            for cast_member in film_info['actors']:
                for film_name, film_info in film_data.items():
                    if 'actors' not in film_info:
                        continue  # tránh KeyError

                    for cast_member in film_info['actors']:
                        # Trường hợp cast_member là dict
                        if isinstance(cast_member, dict):
                            actor_name = cast_member.get('actor_name')

                        # Trường hợp cast_member là list (danh sách tên diễn viên)
                        elif isinstance(cast_member, list):
                            for sub_actor in cast_member:
                                if isinstance(sub_actor, str) and sub_actor not in visited_persons:
                                    new_persons[sub_actor] = None
                            continue

                        # Trường hợp cast_member là string (chỉ là tên)
                        elif isinstance(cast_member, str):
                            actor_name = cast_member

                        # Các trường hợp khác thì bỏ qua
                        else:
                            continue

                        if not actor_name:
                            continue

                        if actor_name not in visited_persons:
                            new_persons[actor_name] = None


                        # Nếu không còn diễn viên mới => dừng
                        if not new_persons:
                            print(" Không còn diễn viên mới để mở rộng.")
                            break

        # Crawl thông tin về các diễn viên mới
        for person in list(new_persons.keys()):
            try:
                print(f" Crawling person: {person}")
                wikitext = get_wikitext(person)
                person_info = extract_infobox_from_wikitext(wikitext)
                film_dict = extract_films_from_person_wikitext(wikitext)
                film_info = get_all_films_from_films_dict(film_dict)
                film_list = []
                for film in film_info:
                    # Ưu tiên các trường phổ biến
                    
                    title = film.get('Tựa đề') or film.get('Tên phim')
                                    
                    # Nếu chưa có tìm key có chứa từ "Tựa"
                    if not title:
                        for key in film.keys():
                            if "Tựa" in key or "Tên" in key:
                                title = film[key]
                                break  # dừng lại khi tìm thấy key đầu tiên
                    if not title:
                        continue

                    # Sau khi có tiêu đề rồi mới kiểm tra có phải phim VN không
                    if not is_vn_film_wikilink(title):
                        continue

                    film_list.append(title)

                person_data[person] = {
                    'info': person_info,
                    'films': film_list
                }

                # Crawl thông tin phim mà diễn viên này tham gia
                for film in film_list:
                    if film not in film_data:
                        print(f" Mở rộng phim mới: {film}")
                        film_wikitext = get_wikitext(film)
                        cast = extract_actors_from_film_wikitext(film_wikitext)
                        director = extract_directors_from_film(film_wikitext)
                        film_info = extract_infobox_from_wikitext(film_wikitext)

                        film_data[film] = {
                            'infobox': film_info,
                            'actors': cast,
                            'director': director
                        }

            except Exception as e:
                print(f" Lỗi khi crawl {person}: {e}")

        visited_persons.update(new_persons.keys())
        current_depth += 1

    print("Mở rộng hoàn tất.")
    return person_data, film_data
persons_data, films_info = expand_network(persons_data, films_info, max_depth=3)



 Đang mở rộng mức 1 ...
 Crawling person: Ngọc Lan (diễn viên)
extract_films_from_subsection: Section không có heading cấp 3
 Crawling person: Lê Bê La
 Mở rộng phim mới: Con đường sáng (phim video)
 Mở rộng phim mới: Tình yêu còn lại
 Mở rộng phim mới: Cuộc phiêu lưu của Hai Lúa
 Mở rộng phim mới: Taxi (phim truyền hình)
 Mở rộng phim mới: Cuộc gọi lúc 0 giờ
 Mở rộng phim mới: Hoa nắng (phim truyền hình Việt Nam)
 Mở rộng phim mới: Cá cược cuộc đời
 Mở rộng phim mới: Thời gian để yêu (phim truyền hình)
 Mở rộng phim mới: Ông trùm (phim truyền hình Việt Nam)
 Mở rộng phim mới: Con gái bố già
 Mở rộng phim mới: Dập tắt lửa lòng
 Mở rộng phim mới: Tiếng sét trong mưa
 Mở rộng phim mới: Không lối thoát (phim truyền hình)
 Mở rộng phim mới: Anh ba khía
 Mở rộng phim mới: Sui gia khắc khẩu
 Mở rộng phim mới: Bão ngầm
 Mở rộng phim mới: Có hẹn với yêu thương
 Mở rộng phim mới: 14 ngày phép
 Mở rộng phim mới: Chuyện ma gần nhà
 Crawling person: Nguyệt Ánh
 Mở rộng phim mới: Miền đất phúc
 Mở

C:\Users\Admin\AppData\Local\Temp\ipykernel_22736\2716991828.py:171: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df.to_dict(orient=orient), df.columns.tolist()


 Mở rộng phim mới: Dưới bầu trời xa cách
 Mở rộng phim mới: Lặng yên dưới vực sâu
 Mở rộng phim mới: Đi qua mùa hạ
 Mở rộng phim mới: Ghét thì yêu thôi
 Mở rộng phim mới: Cô gái nhà người ta
 Mở rộng phim mới: Gặp em ngày nắng
 Mở rộng phim mới: Những chặng đường bụi bặm
 Mở rộng phim mới: Cha tôi người ở lại
 Crawling person: Đình Chiến
extract_films_from_subsection: Section không có heading cấp 3
 Crawling person: Trần Trung
extract_films_from_subsection: Section không có heading cấp 3
 Crawling person: Ngô Thủy Tiên
extract_films_from_subsection: Section không có heading cấp 3
 Crawling person: Mạnh Cường
 Crawling person: Minh Vượng
extract_films_from_subsection: Section không có heading cấp 3
 Crawling person: Hồng Diễm
 Crawling person: Huỳnh Anh diễn viên
extract_films_from_subsection: Section không có heading cấp 3
 Crawling person: Lê Hạ Anh
Bỏ qua 'Phim điện ảnh' (không phải bảng wikitable)
Bỏ qua 'Phim truyền hình' (không phải bảng wikitable)
 Crawling person: Lương Thanh
ex

C:\Users\Admin\AppData\Local\Temp\ipykernel_22736\2716991828.py:171: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df.to_dict(orient=orient), df.columns.tolist()


 Crawling person: Thanh Lam
extract_films_from_subsection: Section không có heading cấp 3
 Crawling person: Gia Khoan
extract_films_from_subsection: Section không có heading cấp 3
 Crawling person: Huang Kiem
extract_films_from_subsection: Section không có heading cấp 3
 Crawling person: Trần Quốc Hùng
extract_films_from_subsection: Section không có heading cấp 3
 Crawling person: Trương Thanh Nghi
extract_films_from_subsection: Section không có heading cấp 3
 Crawling person: Phạm Hải Trường
extract_films_from_subsection: Section không có heading cấp 3
 Crawling person: Trần Lộc
extract_films_from_subsection: Section không có heading cấp 3
 Crawling person: Trần Minh Tuấn diễn viên
extract_films_from_subsection: Section không có heading cấp 3
 Crawling person: Nguyễn Văn Tùng diễn viên
extract_films_from_subsection: Section không có heading cấp 3
 Crawling person: Phạm Duy Hương
extract_films_from_subsection: Section không có heading cấp 3
 Crawling person: Lê Thái Ngọc Hoan
extract_f

In [4]:
from typing import List

from pyparsing import Dict


def clean_text(text: str) -> str:
    """Làm sạch text, loại bỏ markup và templates"""
    if not text or not isinstance(text, str):
        return ""
    
    # Loại bỏ HTML tags
    text = re.sub(r'<br\s*/?>', ', ', text)
    text = re.sub(r'<ref[^>]*>.*?</ref>', '', text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r'<ref[^>]*/?>', '', text)
    text = re.sub(r'<[^>]+>', '', text)
    
    # Loại bỏ Wikipedia links [[link|text]] -> text
    text = re.sub(r'\[\[([^\]|]+)\|([^\]]+)\]\]', r'\2', text)
    text = re.sub(r'\[\[([^\]]+)\]\]', r'\1', text)
    
    # Loại bỏ external links
    text = re.sub(r'\[https?://[^\s\]]+ ([^\]]+)\]', r'\1', text)
    text = re.sub(r'\[https?://[^\]]+\]', '', text)
    
    # Loại bỏ comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Xử lý templates {{...}}
    while '{{' in text:
        match = re.search(r'\{\{([^{}]+)\}\}', text)
        if not match:
            break
        
        template_content = match.group(1)
        replacement = parse_template(template_content)
        text = text[:match.start()] + str(replacement) + text[match.end():]
    
    # Clean up whitespace
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\s*,\s*,\s*', ', ', text)
    text = text.strip(' ,')
    
    return text


def parse_template(template_str: str) -> str:
    """Parse Wikipedia template"""
    if not template_str:
        return ""
    
    parts = [p.strip() for p in template_str.split('|')]
    template_name = parts[0].lower()
    params = parts[1:] if len(parts) > 1 else []
    
    country_codes = {
        'vie': 'Việt Nam',
        'us': 'Hoa Kỳ',
        'nor': 'Na Uy'
    }
    
    if any(keyword in template_name for keyword in ['ngày sinh', 'năm sinh', 'birth date', 'birth year']):
        return parse_birth_date(params)
    elif template_name == 'marriage':
        return parse_marriage(params)
    elif template_name in ['hlist', 'ubl', 'plainlist']:
        return parse_list(params)
    elif template_name == 'convert':
        return parse_convert(params)
    elif template_name == 'height':
        return parse_height(params)
    elif template_name == 'small':
        return f"({params[0]})" if params else ""
    elif template_name in country_codes:
        return country_codes[template_name]
    elif template_name in ['chú thích', 'citation', 'cite']:
        return ""
    
    return ', '.join(params) if params else ""


def parse_birth_date(params: List[str]) -> str:
    try:
        numbers = []
        for p in params:
            matches = re.findall(r'\d+', p)
            numbers.extend(matches)
        
        if len(numbers) >= 3:
            year, month, day = numbers[0], numbers[1], numbers[2]
            return f"{year}-{month.zfill(2)}-{day.zfill(2)}"
        elif len(numbers) >= 1:
            return numbers[0]
    except Exception:
        pass
    return ""


def parse_marriage(params: List[str]) -> str:
    if not params:
        return ""
    spouse = clean_text(params[0])
    if len(params) > 1:
        year_match = re.search(r'\d{4}', params[1])
        if year_match:
            return f"{spouse} ({year_match.group()})"
    return spouse


def parse_list(params: List[str]) -> str:
    items = []
    for param in params:
        cleaned = clean_text(param)
        if cleaned and cleaned not in ['', ' ']:
            items.append(cleaned)
    return ', '.join(items)


def parse_convert(params: List[str]) -> str:
    try:
        value = params[0] if params else ""
        unit = params[1] if len(params) > 1 else ""
        value = re.sub(r'[^\d.,]', '', value)
        if 'm' in unit.lower():
            return f"{value}m"
        return f"{value}"
    except Exception:
        return ""


def parse_height(params: List[str]) -> str:
    for param in params:
        if 'm=' in param:
            height = param.split('=')[1]
            return f"{height}m"
    return ""


def parse_person_info(info_dict: Dict) -> Dict:
    
    if not info_dict:
        return {}
    
    field_mapping = {
        'tên': 'name',
        'name': 'name',
        'tên khai sinh': 'birth_name',
        'tên khác': 'other_names',
        'alias': 'alias',
        'ngày sinh': 'birth_date',
        'birth_date': 'birth_date',
        'nơi sinh': 'birth_place',
        'birth_place': 'birth_place',
        'quốc tịch': 'nationality',
        'dân tộc': 'ethnicity',
        'nghề nghiệp': 'occupation',
        'công việc': 'occupation',
        'occupation': 'occupation',
        'năm hoạt động': 'active_years',
        'năm hoạt động điện ảnh': 'active_years',
        'chiều cao': 'height',
        'học vấn': 'education',
        'education': 'education',
        'người hôn phối': 'spouse',
        'spouse': 'spouse',
        'con cái': 'children',
        'người thân': 'relatives',
        'tác phẩm nổi bật': 'notable_works',
        'hình': 'image',
        'image': 'image'
    }
    
    parsed = {}
    
    for key, value in info_dict.items():
        # Bỏ qua các trường không cần
        if key in ['ghi chú hình', 'signature', 'giải thưởng']:
            continue
        
        field_name = field_mapping.get(key, key)
        parsed_value = clean_text(str(value))
        
        if parsed_value:
            parsed[field_name] = parsed_value
    
    return parsed


def parse_persons_data(persons_data: Dict) -> Dict:
    
    parsed_persons = {}
    
    for person_name, person_data in persons_data.items():
        # Parse info
        raw_info = person_data.get('info', {})
        parsed_info = parse_person_info(raw_info)
        
        # Giữ nguyên films list
        films = person_data.get('films', [])
        
        parsed_persons[person_name] = {
            'info': parsed_info,
            'films': films
        }
    
    return parsed_persons


def parse_film_info(infobox_dict: Dict) -> Dict:
    
    if not infobox_dict:
        return {}
    
    field_mapping = {
        'tên phim': 'title',
        'tựa đề': 'title',
        'năm phát hành': 'year',
        'năm': 'year',
        'thể loại': 'genre',
        'đạo diễn': 'director',
        'diễn viên': 'cast',
        'nhà sản xuất': 'producer',
        'thời lượng': 'duration',
        'quốc gia': 'country',
        'ngôn ngữ': 'language',
        'kinh phí': 'budget',
        'doanh thu': 'box_office'
    }
    
    parsed = {}
    
    for key, value in infobox_dict.items():
        # Bỏ qua các trường không cần
        if key in ['hình', 'ghi chú hình']:
            continue
        
        field_name = field_mapping.get(key.lower(), key)
        parsed_value = clean_text(str(value))
        
        if parsed_value:
            parsed[field_name] = parsed_value
    
    return parsed


def parse_films_data(films_data: Dict) -> Dict:
    
    parsed_films = {}
    
    for film_name, film_data in films_data.items():
        try:
            # Parse infobox
            raw_infobox = film_data.get('infobox', {})
            if raw_infobox is None:
                raw_infobox = {}
            parsed_infobox = parse_film_info(raw_infobox)
            
            # Parse actors (clean character names)
            actors = film_data.get('actors', [])
            if actors is None:
                actors = []
            
            parsed_actors = []
            for actor in actors:
                if isinstance(actor, dict):
                    parsed_actor = {
                        'actor_name': actor.get('actor_name', ''),
                        'character': clean_text(str(actor.get('character', ''))) if actor.get('character') else ''
                    }
                    parsed_actors.append(parsed_actor)
            
            # Parse director (clean names) - XỬ LÝ TẤT CẢ TRƯỜNG HỢP
            directors = film_data.get('director', [])
            parsed_directors = []
            
            if directors is None:
                # Nếu là None, để list rỗng
                parsed_directors = []
            elif isinstance(directors, str):
                # Nếu là string đơn lẻ
                cleaned = clean_text(directors)
                if cleaned:
                    parsed_directors = [cleaned]
            elif isinstance(directors, list):
                # Nếu là list
                for d in directors:
                    if d is not None:
                        # Có thể là dict hoặc string
                        if isinstance(d, dict):
                            director_name = d.get('director_name', '')
                        else:
                            director_name = str(d)
                        
                        cleaned = clean_text(director_name)
                        if cleaned:
                            parsed_directors.append(cleaned)
            
            parsed_films[film_name] = {
                'title': film_data.get('title', film_name),
                'infobox': parsed_infobox,
                'actors': parsed_actors,
                'director': parsed_directors
            }
            
        except Exception as e:
            print(f"⚠️  Lỗi khi parse phim '{film_name}': {e}")
            # Vẫn thêm phim nhưng với dữ liệu tối thiểu
            parsed_films[film_name] = {
                'title': film_name,
                'infobox': {},
                'actors': [],
                'director': []
            }
    
    return parsed_films



persons_data = parse_persons_data(persons_data)
films_info = parse_films_data(films_info)

import networkx as nx

# xay do thi 2 phia: nguoi va phim
def build_bipartite_graph(persons_data, films_data):
    
    B = nx.Graph()

    # Thêm diễn viên / đạo diễn
    for person_name, person_data in persons_data.items():
        B.add_node(person_name, type='person', info=person_data.get('info', {}))

    # Thêm phim
    for film_name, film_data in films_data.items():
        B.add_node(film_name, type='film', infobox=film_data.get('infobox', {}))

    # Thêm các cạnh
    actor_edges = 0
    director_edges = 0

    for film_name, film_data in films_data.items():
        # dien vien
        actors = film_data.get('actors', [])  # Lấy danh sách actors một lần
        
        for cast_member in actors:
            # XỬ LÝ CẤU TRÚC LỒNG NHAU
            
            # Trường hợp 1: cast_member là list chứa dict
            if isinstance(cast_member, list):
                for item in cast_member:
                    if isinstance(item, dict):
                        actor_name = item.get('actor_name')
                        character = item.get('character', '')
                        
                        if actor_name and actor_name.strip():
                            actor_name = actor_name.strip()
                            if actor_name in B.nodes:
                                B.add_edge(actor_name, film_name, 
                                         role='actor', 
                                         character=character)
                                actor_edges += 1
                    
                    elif isinstance(item, str):
                        # List chứa string
                        actor_name = item.strip()
                        if actor_name and actor_name in B.nodes:
                            B.add_edge(actor_name, film_name, 
                                     role='actor', 
                                     character='')
                            actor_edges += 1
            
            # Trường hợp 2: cast_member là dict trực tiếp
            elif isinstance(cast_member, dict):
                actor_name = cast_member.get('actor_name')
                character = cast_member.get('character', '')
                
                if actor_name and actor_name.strip():
                    actor_name = actor_name.strip()
                    if actor_name in B.nodes:
                        B.add_edge(actor_name, film_name, 
                                 role='actor', 
                                 character=character)
                        actor_edges += 1
            
            # Trường hợp 3: cast_member là string
            elif isinstance(cast_member, str):
                actor_name = cast_member.strip()
                if actor_name and actor_name in B.nodes:
                    B.add_edge(actor_name, film_name, 
                             role='actor',  
                             character='')
                    actor_edges += 1

        # dao dien
        director = film_data.get('director')
        if director:
            # co the la str hoac list
            if isinstance(director, str):
                directors = [director]
            elif isinstance(director, list):
                directors = director
            else:
                directors = []

            for dir_name in directors:
                if dir_name and dir_name in B.nodes:
                    B.add_edge(dir_name, film_name, role='director')
                    director_edges += 1

    print(f"Graph built successfully: {B.number_of_nodes()} nodes, {B.number_of_edges()} edges.")
    print(f"{actor_edges} actor edges, {director_edges} director edges.")

    return B

# Sử dụng
B = build_bipartite_graph(persons_data, films_info)

Graph built successfully: 3629 nodes, 8166 edges.
8079 actor edges, 148 director edges.


In [5]:
import networkx as nx
from collections import defaultdict
# xay do thi mqh giua nguoi va nguoi
def build_collaboration_graph(bipartite_graph):
   
    G = nx.Graph()

    # Lấy danh sách người
    people = [n for n, attr in bipartite_graph.nodes(data=True) if attr['type'] == 'person']

    # Thêm nút người
    for person in people:
        G.add_node(person, **bipartite_graph.nodes[person])

    # Dùng defaultdict để lưu thông tin hợp tác
    collaborations = {}

    # Duyệt qua từng phim trong đồ thị hai phía
    films = [n for n, attr in bipartite_graph.nodes(data=True) if attr['type'] == 'film']

    for film in films:
        # Lấy tất cả người tham gia phim đó
        film_people = [n for n in bipartite_graph.neighbors(film) if bipartite_graph.nodes[n]['type'] == 'person']

        # Tạo cặp hợp tác giữa các người cùng tham gia phim
        for i in range(len(film_people)):
            for j in range(i + 1, len(film_people)):
                p1, p2 = sorted([film_people[i], film_people[j]])
                if (p1, p2) not in collaborations:
                    collaborations[(p1, p2)] = {
                        'weight': 0,
                        'films': [],
                        'roles': [],
                        'same school': False,
                        'school': None,
                        'same location': False,
                        'location': None
                    }
                role1 = bipartite_graph[p1][film]['role']
                role2 = bipartite_graph[p2][film]['role']

                collaborations[(p1, p2)]['weight'] += 1
                collaborations[(p1, p2)]['films'].append(film)
                collaborations[(p1, p2)]['roles'].append(f"{role1}-{role2}")
                
    # them quan he cung truong, cung que
    for i in range(len(people)):
        for j in range(i + 1, len(people)):
            p1, p2 = sorted([people[i], people[j]])
            
            # Lấy thông tin của 2 người
            info1 = bipartite_graph.nodes[p1].get('info', {})
            info2 = bipartite_graph.nodes[p2].get('info', {})
            
            # Kiểm tra cùng trường
            school1 = info1.get('education') or info1.get('học vấn')
            school2 = info2.get('education') or info2.get('học vấn')
            
            same_school = False
            school_name = None
            if school1 and school2:
                school1 = school1.strip()
                school2 = school2.strip()
                if school1 == school2:
                    same_school = True
                    school_name = school1
            
            # Kiểm tra cùng quê
            loc1 = info1.get('birth_place') or info1.get('nơi sinh')
            loc2 = info2.get('birth_place') or info2.get('nơi sinh')
            
            same_location = False
            location_name = None
            if loc1 and loc2:
                loc1 = loc1.strip()
                loc2 = loc2.strip()
                if loc1 == loc2:
                    same_location = True
                    location_name = loc1
            
            # Nếu có bất kỳ quan hệ nào
            if same_school or same_location or (p1, p2) in collaborations:
                # Khởi tạo nếu chưa có (trường hợp chỉ cùng trường/quê, không cùng phim)
                if (p1, p2) not in collaborations:
                    collaborations[(p1, p2)] = {
                        'weight': 0,
                        'films': [],
                        'roles': [],
                        'same_school': False,
                        'school': None,
                        'same_location': False,
                        'location': None
                    }
                
                # Cập nhật thông tin
                if same_school:
                    collaborations[(p1, p2)]['same_school'] = True
                    collaborations[(p1, p2)]['school'] = school_name
                
                if same_location:
                    collaborations[(p1, p2)]['same_location'] = True
                    collaborations[(p1, p2)]['location'] = location_name

    # 7. Thêm edges vào graph
    for (p1, p2), data in collaborations.items():
        weight = 0
        film_count = data.get('weight', 0)
        weight += film_count
        if data.get('same_school'):
            weight += 0.5
        if data.get('same_location'):
            weight += 0.3
        
        G.add_edge(
            p1, p2,
            film_count=data.get('weight', 0),
            films=data.get('films', []),
            collaboration_types=list(set(data.get('roles', []))),
            same_school=data.get('same_school', False),
            school=data.get('school'),
            same_location=data.get('same_location', False),
            location=data.get('location'),
            weight=weight  
        )

    return G

G_bipartite = build_bipartite_graph(persons_data, films_info)
G_actor_collab = build_collaboration_graph(G_bipartite)
print(f"Số nút trong đồ thị cộng tác diễn viên: {G_actor_collab.number_of_nodes()}")
print(f"Số cạnh trong đồ thị cộng tác diễn viên: {G_actor_collab.number_of_edges()}")
print(f"So node trong do thi bipartite: {G_bipartite.number_of_nodes()}")
print(f"So canh trong do thi bipartite: {G_bipartite.number_of_edges()}")





Graph built successfully: 3629 nodes, 8166 edges.
8079 actor edges, 148 director edges.
Số nút trong đồ thị cộng tác diễn viên: 2985
Số cạnh trong đồ thị cộng tác diễn viên: 84418
So node trong do thi bipartite: 3629
So canh trong do thi bipartite: 8166
